# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [22]:
# imports
from openai import OpenAI
import os
from IPython.display import Markdown, display, update_display



In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


In [24]:
# set up environment
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

API key looks good so far


In [25]:
#accepts the user input and the agent behaves like that
def create_dynamic_system_prompt(agent_type):
    return f"You are an expert {agent_type} and helpful assistant that can answer questions and help with tasks. Explain in detail about the whole process needed to achieve the task"    

In [26]:
# here is the question; type over this to ask something new


question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [27]:
# Get gpt-4o-mini to answer, with streaming
def askGptAgent(type):

    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": create_dynamic_system_prompt(type)},
            {"role": "user", "content": question}
        ],
        stream=True
    )
    return stream


In [28]:
# Get Llama 3.2 to answer
def askLLamaAgent(type):

    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": create_dynamic_system_prompt(type)},
            {"role": "user", "content": question}
        ],
        stream=True
    )
    return stream

In [29]:
def askCustomAgent(agent_type, model):
    print(f"Model used is {model}")
    print(f"Agent type is {agent_type}")
    print(f"System prompt is {create_dynamic_system_prompt(agent_type)}")
    if model == "gpt":
        return askGptAgent(agent_type)
    elif model == "llama":
        return askLLamaAgent(agent_type)
    else:
        print("Invalid model")
        return None

In [30]:
def displayResult(stream):
    display_handle = display(Markdown(""), display_id=True)

    response = "" # Initialize response as an empty string
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [31]:
stream = askCustomAgent("physicist", "gpt")
displayResult(stream)

Model used is gpt
Agent type is physicist
System prompt is You are an expert physicist and helpful assistant that can answer questions and help with tasks. Explain in detail about the whole process needed to achieve the task


The code snippet you've provided is a Python expression that uses a combination of a generator, set comprehension, and the `yield from` statement. Let's break it down step by step to understand what it does and why it is structured this way.

### Components of the Code

1. **Set Comprehension**:
    - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
    - It creates a set—a collection of unique items—by iterating over a list or iterable called `books`.
    - For each `book` in `books`, it attempts to retrieve the value of the key `"author"` using `book.get("author")`.
    - The condition `if book.get("author")` ensures that only books where the `"author"` key exists and has a truthy value (not `None`, `False`, `0`, `""`, or similar) are included in the set.

2. **Yield from**:
    - The `yield from` statement is used in generator functions. It allows the function to yield all values from another iterable (in this case, the set created by the set comprehension).
    - This means that when the generator function containing this line is called, it will yield each unique author from the set, effectively yielding each of the unique authors found in the `books`.

### Explanation of the Whole Process

1. **Input**:
   - The function that contains this code presumably receives a list of dictionaries named `books`. Each dictionary represents a book and might have various keys, one of which is `"author"`.

2. **Execution of Set Comprehension**:
   - The code iterates over each dictionary (`book`) in the `books` list.
   - It checks for the presence of an `"author"` key:
     - If the author exists and is truthy, its value is added to a unique collection (set).
   - The use of a set guarantees that even if multiple books have the same author, that author's name will only appear once in the final collection.

3. **Yielding Authors**:
   - The `yield from` statement then takes the set resulting from the comprehension and yields each unique author one by one.
   - This allows the calling function or whichever part of the program is consuming this generator to iterate over the authors as needed, without needing to create a separate list or loop through the set manually.

### Example

Here's a quick example to illustrate how the code works:

```python
books = [
    {"title": "Book One", "author": "Alice"},
    {"title": "Book Two", "author": "Bob"},
    {"title": "Book Three", "author": "Alice"},
    {"title": "Book Four"},  # No author
    {"title": "Book Five", "author": None},  # Non-truthy author
]

def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

for author in unique_authors(books):
    print(author)
```

### Output:
```
Alice
Bob
```

In this example:
- The set comprehension collects the unique authors "Alice" and "Bob" while ignoring the entries without valid authors.
- The `yield from` statement allows iterating through the distinct authors easily, producing a sequence of author names.

### Conclusion
This code snippet is a concise and effective way to collect and yield unique author names from a list of book dictionaries, showcasing the use of set comprehensions and generator functions in Python effectively.

In [ ]:
#call the agent to act like an expert in field
